# MLflow Training Tutorial

This `train.pynb` Jupyter notebook predicts the quality of wine using [sklearn.linear_model.ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html).

In [3]:
# %env http_proxy=http://proxy.tmi.alcf.anl.gov:3128
# %env https_proxy=http://proxy.tmi.alcf.anl.gov:3128

In [1]:
# Wine Quality Sample
def train(in_alpha, in_l1_ratio):
    import os
    import warnings
    import sys

    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet
    from mlflow.models.signature import infer_signature
    from urllib.parse import urlparse


    import mlflow
    import mlflow.sklearn
    
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)
    
    
    mlflow.set_tracking_uri("http://127.0.0.1:8080") # set tracking server information
    mlflow.set_experiment("wine_quality_linear_regression") # set experiment name
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    np.random.seed(40)

    # Read the wine-quality csv file
    #     csv_url ='http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    #     try:
    #         data = pd.read_csv(csv_url, sep=';')
    #     except Exception as e:
    #         logger.exception(
    #             "Unable to download training & test CSV, check your internet connection. Error: %s", e)

    # Split the data into training and test sets. (0.75, 0.25) split.
    data = pd.read_csv('winequality-red.csv', sep=';')
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha and no l1_ratio is provided
    alpha = float(in_alpha) if in_alpha else 0.5
    l1_ratio = float(in_l1_ratio) if in_l1_ratio else 0.5

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        mlflow.autolog() # automatically logs metrics 
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        # Model registry does not work with file store
        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        signature = infer_signature(train_x, lr.predict(test_x))
        mlflow.sklearn.log_model(
            sk_model=lr,
            artifact_path="sklearn-model",
            registered_model_name="sk-learn-random-forest-reg-model-test",
            signature=signature
        )


In [2]:
train(0.5, 0.5)

2022/05/19 12:04:20 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/19 12:04:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
2022/05/19 12:04:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-random-forest-reg-model, version 20
Created version '20' of model 'sk-learn-random-forest-reg-model'.


In [9]:
train(0.1, 0.1)

2022/05/19 00:50:44 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/19 00:50:44 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7128829045893679
  MAE: 0.5462202174984664
  R2: 0.2799376066653344


Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
2022/05/19 00:50:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-random-forest-reg-model, version 12
Created version '12' of model 'sk-learn-random-forest-reg-model'.


In [2]:
train(1.2, 0.9)

2022/05/19 02:09:22 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/19 02:09:23 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/19 02:09:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/projects/datascience/atanikanti/mlflow_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


Elasticnet model (alpha=1.200000, l1_ratio=0.900000):
  RMSE: 0.832793747039262
  MAE: 0.6731067319470196
  R2: 0.017327866377111367


Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
2022/05/19 02:09:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-random-forest-reg-model, version 18
Created version '18' of model 'sk-learn-random-forest-reg-model'.


In [2]:
train(1.8, 0.2)

2022/05/19 11:25:48 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2022/05/19 11:25:54 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/19 11:26:00 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/projects/datascience/atanikanti/mlflow_env/lib/python3.9/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


Elasticnet model (alpha=1.800000, l1_ratio=0.200000):
  RMSE: 0.8318093095121921
  MAE: 0.6648769121659138
  R2: 0.019649707922284376


Registered model 'sk-learn-random-forest-reg-model' already exists. Creating a new version of this model...
2022/05/19 11:26:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: sk-learn-random-forest-reg-model, version 20
Created version '20' of model 'sk-learn-random-forest-reg-model'.


In [5]:
%run train.py 0.5 1

Elasticnet model (alpha=0.500000, l1_ratio=1.000000):
  RMSE: 0.832819092896359
  MAE: 0.6681279771237894
  R2: 0.017268050734704055
